In [31]:
import json
import pandas as pd
import dill as pickle

class DataUtil2:
    """"""
    @classmethod
    def load(cls, file, format, date_cols= None):
        """"""
        data = ''
        if(format== 'csv'):
            data = pd.read_csv(file, parse_dates= date_cols)
        elif(format== 'json'):
            with open(file, 'r') as i_file:
                data = json.load(file)
            i_file.close()
        elif(format== 'pkl'):
            with open(file, 'rb') as i_file:
                data = pickle.load(i_file)
            i_file.close()
        elif(format == 'hdf'):
            data = pd.read_hdf(path_or_buf= file, key='undefined')

        return  data

    @classmethod
    def save(cls, data, file, format, precision= 8):
        """"""
        if(format == 'csv'):
            data.to_csv(file, float_format= '%%.%df' % precision, index= False)
        elif(format == 'json'):
            with open(file, 'w') as o_file:
                json.dump(data, o_file, ensure_ascii= True, indent= 4)
            o_file.close()
        elif(format == 'pkl'):
            with open(file, 'wb') as o_file:
                pickle.dump(data, o_file, -1)
            o_file.close()
        elif(format== 'hdf'):
            data.to_hdf(path_or_buf= file, key='undefined', mode='w', complib='blosc')

        return

In [11]:
#
from sklearn import *
import numpy as np
import pandas as pd
import datetime
import time
import os,sys

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5

DataBaseDir = '../../data'
InputDir = '%s/l0/kfold' % DataBaseDir

kfold = 5
strategy = 'en'
drop_cols = ['id', 'air_store_id', 'visit_date', 'visitors', 'hpg_store_id', 'fold']

valid_dfs = []
for fold in range(kfold):
    FoldInputDir = '%s/%s' % (InputDir, fold)
    valid = pd.read_csv('%s/valid.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
#     holdout = pd.read_csv('%s/holdout.csv' % FoldInputDir, parse_dates= ['visit_date']).reset_index(drop= True)
    valid = valid[valid['visit_date'] > datetime.datetime(2016, 7, 1)]
#     holdout = holdout[holdout['visit_date'] > datetime.datetime(2016, 7, 1)]
#     print('fold %s, len %s' % (fold, len(valid)))
    valid['fold'] = fold
    valid_dfs.append(valid)
DataSet = pd.concat(valid_dfs, axis= 0, ignore_index= True)
print('load data done.')

print(len(DataSet[DataSet['air_reserved_visitors'] > 0]))
print(len(DataSet[DataSet['hpg_reserved_visitors'] > 0]))
print(DataSet[DataSet['air_reserved_visitors'] > 0][['air_reserved_visitors', 'hpg_reserved_visitors', 'reserved_visitors']])
sys.exit(1)
##
params = {
    'alpha': [400, 40, 4, 4e-1, 4e-2],
    'l1_ratio': [0.02, 0.2, 0.8, 1.0]
}

cnt = 0
groupped = DataSet.groupby(['air_store_id', 'holiday_flg'])
evals = {}
start = time.time()
for g in groupped.groups:
    if(cnt == 10):
        break
    TrainData = groupped.get_group(g)
    # score
    BestFit = {
        'cv': {'score': 100, 'params': {'alpha': -1, 'l1_ratio': -1}},
        'dummy': {'score': 100, 'params': {'alpha': -1, 'l1_ratio': -1}}
    }
    for al in params['alpha']:
        for lr in params['l1_ratio']:
            cv_score = .0
            cv_dummy = .0
            for fold in range(kfold):
                FoldData = {
                    'train': TrainData[TrainData['fold'] != fold],
                    'valid': TrainData[TrainData['fold'] == fold]
                }
                reserve_cols = [c for c in FoldData['train'].columns if('reserve' in c)]
                col = [c for c in FoldData['train'].columns if((c not in drop_cols) and (c not in reserve_cols))]
                # train
                model = linear_model.ElasticNet(alpha= al, l1_ratio= lr, max_iter= 200, tol= 1e-4, selection= 'cyclic', random_state= 2017)
                model.fit(FoldData['train'][col].astype(np.float32, copy=False), FoldData['train']['visitors'].values.astype(np.float32, copy=False))
                # for valid
                FoldData['valid'][strategy] = model.predict(FoldData['valid'][col])
                FoldData['valid']['dummy'] = FoldData['valid']['visitors'].mean()
                rmsle_valid = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid'][strategy])
                rmsle_dummy = RMSLE(FoldData['valid']['visitors'].values, FoldData['valid']['dummy'])
                cv_score += rmsle_valid
                cv_dummy += rmsle_dummy
            cv_score /= kfold
            cv_dummy /= kfold
            if(BestFit['cv']['score'] > cv_score):
                BestFit['cv']['score'] = cv_score
                BestFit['cv']['params']['alpha'] = al
                BestFit['cv']['params']['l1_ratio'] = lr
            if(BestFit['dummy']['score'] > cv_dummy):
                BestFit['dummy']['score'] = cv_dummy
                BestFit['dummy']['params']['alpha'] = al
                BestFit['dummy']['params']['l1_ratio'] = lr
    evals['air_store_id'] = g[0]
    evals['is_weekends'] = g[1]
    evals['lift'] = BestFit['dummy']['score'] - BestFit['cv']['score']
    print(g)
    print(BestFit)
    cnt += 1
    end = time.time()
    print('time elapsed %.2fs' % (end - start))
end = time.time()
eval_df = pd.DataFrame(data= evals, index= range(len(evals)))
print('mean lift %.4f, time elapsed %s.2fs' % (eval_df['lift'].mean(), (end - start)))

load data done.
20687
10271
        air_reserved_visitors  hpg_reserved_visitors  reserved_visitors
538                  1.945910               0.000000           0.000000
540                  1.098612               0.000000           0.000000
541                  1.386294               0.000000           0.000000
543                  1.098612               0.000000           0.000000
547                  1.609438               0.000000           0.000000
549                  2.197225               0.000000           0.000000
550                  2.833213               0.000000           0.000000
551                  1.609438               0.000000           0.000000
554                  1.945910               0.000000           0.000000
555                  2.890372               0.000000           0.000000
556                  2.302585               0.000000           0.000000
557                  1.945910               0.000000           0.000000
558                  1.609438       

SystemExit: 1

/Users/yuanpingzhou/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
